                                          VIEW CELLS IN A MINIMIZED VIEW FOR A BETTER VISUAL OF THE DATAFRAMES.


In [0]:
spark.conf.set("fs.azure.account.key.projectstorageaccount11.dfs.core.windows.net", "RKyEVqMj4IcV+b9I73Rm4BpdXtYoeguqN8XIlws7EQcJHx6J0wp1o3RJZtPBdB94E3uWlAeEak7I+AStIY2wlQ==")

In [0]:
dbutils.fs.ls("abfss://raw@projectstorageaccount11.dfs.core.windows.net/")


[FileInfo(path='abfss://raw@projectstorageaccount11.dfs.core.windows.net/Amir-Razvi/', name='Amir-Razvi/', size=0, modificationTime=1719871925000)]

In [0]:
from pyspark.sql import SparkSession
from datetime import datetime

spark = SparkSession.builder.appName("ReadData").getOrCreate()
storage_account_name = "projectstorageaccount11"
src_container = "raw"
file_path = "Amir-Razvi/Health_Insights/main/healthcare_dataset.csv"
target_container = "cleaned"

adls_url = f"abfss://{src_container}@{storage_account_name}.dfs.core.windows.net/{file_path}"

df = spark.read.csv(adls_url, sep=',', header=True,inferSchema=True)

df.show()

df.createOrReplaceTempView("view_healthdataset")

+-------------------+---+------+----------+-----------------+-----------------+----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|               Name|Age|Gender|Blood Type|Medical Condition|Date of Admission|          Doctor|            Hospital|Insurance Provider|    Billing Amount|Room Number|Admission Type|Discharge Date| Medication|Test Results|
+-------------------+---+------+----------+-----------------+-----------------+----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|      Bobby JacksOn| 30|  Male|        B-|           Cancer|       2024-01-31|   Matthew Smith|     Sons and Miller|        Blue Cross|18856.281305978155|        328|        Urgent|    2024-02-02|Paracetamol|      Normal|
|       LesLie TErRy| 62|  Male|        A+|          Obesity|       2019-08-20| Samantha Davies|            

In [0]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Blood Type: string (nullable = true)
 |-- Medical Condition: string (nullable = true)
 |-- Date of Admission: date (nullable = true)
 |-- Doctor: string (nullable = true)
 |-- Hospital: string (nullable = true)
 |-- Insurance Provider: string (nullable = true)
 |-- Billing Amount: double (nullable = true)
 |-- Room Number: integer (nullable = true)
 |-- Admission Type: string (nullable = true)
 |-- Discharge Date: date (nullable = true)
 |-- Medication: string (nullable = true)
 |-- Test Results: string (nullable = true)



In [0]:
from pyspark.ml.feature import Imputer
imputer=Imputer(inputCols=['Age','Billing Amount'],outputCols=['Age_Cleaned','BillingAmount_Cleaned'])

df_imputed=imputer.fit(df).transform(df)
df_imputed.show()

+-------------------+---+------+----------+-----------------+-----------------+----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+-----------+---------------------+
|               Name|Age|Gender|Blood Type|Medical Condition|Date of Admission|          Doctor|            Hospital|Insurance Provider|    Billing Amount|Room Number|Admission Type|Discharge Date| Medication|Test Results|Age_Cleaned|BillingAmount_Cleaned|
+-------------------+---+------+----------+-----------------+-----------------+----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+-----------+---------------------+
|      Bobby JacksOn| 30|  Male|        B-|           Cancer|       2024-01-31|   Matthew Smith|     Sons and Miller|        Blue Cross|18856.281305978155|        328|        Urgent|    2024-02-02|Paracetamol|      Normal|       

In [0]:

#checking if any duplicates exist
total_rows=df_imputed.count()
extra_rows=df_imputed.distinct().count()

print(f"Total rows: {total_rows}")
print(f"Extra rows: {extra_rows}")

if total_rows > extra_rows:
    print(f"There are {total_rows - extra_rows} duplicate rows.")
else:
    print("There are no duplicate rows.")

Total rows: 55500
Extra rows: 54966
There are 534 duplicate rows.


In [0]:
df_no_duplicates=df_imputed.dropDuplicates()
df_no_duplicates.show()

+-------------------+---+------+----------+-----------------+-----------------+----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+-----------+---------------------+
|               Name|Age|Gender|Blood Type|Medical Condition|Date of Admission|          Doctor|            Hospital|Insurance Provider|    Billing Amount|Room Number|Admission Type|Discharge Date| Medication|Test Results|Age_Cleaned|BillingAmount_Cleaned|
+-------------------+---+------+----------+-----------------+-----------------+----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+-----------+---------------------+
|     aaRon MARtiNeZ| 38|Female|        A-|     Hypertension|       2023-08-13|    Douglas Mayo|         Lyons-Blair|          Medicare| 7999.586879604188|        288|        Urgent|    2023-09-05|    Lipitor|Inconclusive|       

In [0]:
#checking if any duplicates exist
total_rows=df_no_duplicates.count()
extra_rows=df_no_duplicates.distinct().count()

print(f"Total rows: {total_rows}")
print(f"Extra rows: {extra_rows}")

if total_rows > extra_rows:
    print(f"There are {total_rows - extra_rows} duplicate rows.")
else:
    print("There are no duplicate rows.")

Total rows: 54966
Extra rows: 54966
There are no duplicate rows.


In [0]:
df_no_duplicates.createOrReplaceTempView("view_healthdataset_cleaned")

In [0]:
checking = """
                SELECT Name, Age_Cleaned, Gender, `Blood Type`, `Medical Condition`, `Date of Admission`, Hospital, `Insurance Provider`,`BillingAmount_Cleaned`,`Admission Type`,`Discharge Date`, Medication,`Test Results`
                FROM view_healthdataset_cleaned
              """

output_df_cleaned = spark.sql(checking)
output_df_cleaned.show(10)
#analyzing which all columns we do not need and excluding them

+----------------+-----------+------+----------+-----------------+-----------------+--------------------+------------------+---------------------+--------------+--------------+-----------+------------+
|            Name|Age_Cleaned|Gender|Blood Type|Medical Condition|Date of Admission|            Hospital|Insurance Provider|BillingAmount_Cleaned|Admission Type|Discharge Date| Medication|Test Results|
+----------------+-----------+------+----------+-----------------+-----------------+--------------------+------------------+---------------------+--------------+--------------+-----------+------------+
|ChRISTopher BerG|         58|Female|       AB-|           Cancer|       2021-05-23|      Padilla-Walker|  UnitedHealthcare|    19784.63106221073|      Elective|    2021-06-22|Paracetamol|Inconclusive|
|    andrEw waTtS|         28|Female|        O+|         Diabetes|       2020-11-18|Hernandez Rogers ...|          Medicare|    37909.78240987528|      Elective|    2020-12-18|  Ibuprofen|    

In [0]:


output_df_cleaned.write.mode('overwrite').csv(f"abfss://{target_container}@projectstorageaccount11.dfs.core.windows.net/cleaned_healthdata",header=True)


                                                                                  DATA IS NOW CLEANED

In [0]:


csvPath = "wasbs://cleaned@projectstorageaccount11.blob.core.windows.net/cleaned_healthdata/part-000*"

# Read all CSV files matching the pattern into a DataFrame
new_df = spark.read.format("csv").option("header", "true").load(csvPath)

In [0]:
new_df.show(10)

+-------------------+-----------+------+----------+-----------------+-----------------+--------------------+------------------+---------------------+--------------+--------------+-----------+------------+
|               Name|Age_Cleaned|Gender|Blood Type|Medical Condition|Date of Admission|            Hospital|Insurance Provider|BillingAmount_Cleaned|Admission Type|Discharge Date| Medication|Test Results|
+-------------------+-----------+------+----------+-----------------+-----------------+--------------------+------------------+---------------------+--------------+--------------+-----------+------------+
|JaCQueLiNE maRshAll|         27|  Male|        A+|     Hypertension|       2024-02-22|        Myers-Monroe|  UnitedHealthcare|    39615.60065570468|        Urgent|    2024-02-24| Penicillin|Inconclusive|
|      RoBErT sExtON|         54|  Male|        A-|           Cancer|       2020-11-19|    Sherman-Martinez|        Blue Cross|    3771.237165573879|        Urgent|    2020-11-27| 

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Define a function to standardize the spelling
def standardize_name(name):
    return ' '.join(word.capitalize() for word in name.lower().split())

# Register the function as a UDF (User Defined Function)
standardize_name_udf = udf(standardize_name, StringType())

# Apply the UDF to the Name column
new_df = new_df.withColumn("Name", standardize_name_udf("Name"))

new_df.show(10)


+-------------------+-----------+------+----------+-----------------+-----------------+--------------------+------------------+---------------------+--------------+--------------+-----------+------------+
|               Name|Age_Cleaned|Gender|Blood Type|Medical Condition|Date of Admission|            Hospital|Insurance Provider|BillingAmount_Cleaned|Admission Type|Discharge Date| Medication|Test Results|
+-------------------+-----------+------+----------+-----------------+-----------------+--------------------+------------------+---------------------+--------------+--------------+-----------+------------+
|Jacqueline Marshall|         27|  Male|        A+|     Hypertension|       2024-02-22|        Myers-Monroe|  UnitedHealthcare|    39615.60065570468|        Urgent|    2024-02-24| Penicillin|Inconclusive|
|      Robert Sexton|         54|  Male|        A-|           Cancer|       2020-11-19|    Sherman-Martinez|        Blue Cross|    3771.237165573879|        Urgent|    2020-11-27| 

In [0]:
new_df.createOrReplaceTempView("view_healthdataset_processed")

In [0]:
final_checking_test = """
                SELECT Name
                FROM view_healthdataset_processed
              """

output_df_final_test = spark.sql(final_checking_test)
output_df_final_test.show(10)
#Standardized Names

+-------------------+
|               Name|
+-------------------+
|Jacqueline Marshall|
|      Robert Sexton|
|    Michael Ellison|
|       Larry Ramsey|
|      Rebecca Moore|
|        Jimmy Jones|
|      Jennifer West|
|     Melissa Morgan|
|      Michael Riley|
|   Dr. Kevin Hayden|
+-------------------+
only showing top 10 rows



In [0]:
from pyspark.sql.functions import monotonically_increasing_id

# Add a new column named "ID" with unique IDs
output_df_final = new_df.withColumn("ID", monotonically_increasing_id()+1)

# Reorder the columns with ID at the beginning
output_df_final = output_df_final.select("ID", *output_df_final.columns[:-1])

# Show the DataFrame with IDs
output_df_final.show(10)


+---+-------------------+-----------+------+----------+-----------------+-----------------+--------------------+------------------+---------------------+--------------+--------------+-----------+------------+
| ID|               Name|Age_Cleaned|Gender|Blood Type|Medical Condition|Date of Admission|            Hospital|Insurance Provider|BillingAmount_Cleaned|Admission Type|Discharge Date| Medication|Test Results|
+---+-------------------+-----------+------+----------+-----------------+-----------------+--------------------+------------------+---------------------+--------------+--------------+-----------+------------+
|  1|Jacqueline Marshall|         27|  Male|        A+|     Hypertension|       2024-02-22|        Myers-Monroe|  UnitedHealthcare|    39615.60065570468|        Urgent|    2024-02-24| Penicillin|Inconclusive|
|  2|      Robert Sexton|         54|  Male|        A-|           Cancer|       2020-11-19|    Sherman-Martinez|        Blue Cross|    3771.237165573879|        Urg

In [0]:
output_df_final.show(100)

+---+-------------------+-----------+------+----------+-----------------+-----------------+--------------------+------------------+---------------------+--------------+--------------+-----------+------------+
| ID|               Name|Age_Cleaned|Gender|Blood Type|Medical Condition|Date of Admission|            Hospital|Insurance Provider|BillingAmount_Cleaned|Admission Type|Discharge Date| Medication|Test Results|
+---+-------------------+-----------+------+----------+-----------------+-----------------+--------------------+------------------+---------------------+--------------+--------------+-----------+------------+
|  1|Jacqueline Marshall|         27|  Male|        A+|     Hypertension|       2024-02-22|        Myers-Monroe|  UnitedHealthcare|    39615.60065570468|        Urgent|    2024-02-24| Penicillin|Inconclusive|
|  2|      Robert Sexton|         54|  Male|        A-|           Cancer|       2020-11-19|    Sherman-Martinez|        Blue Cross|    3771.237165573879|        Urg

In [0]:
output_df_final = output_df_final.withColumnRenamed("Medical Condition", "Medical_Condition")

output_df_final.show()


+---+-------------------+-----------+------+----------+-----------------+-----------------+--------------------+------------------+---------------------+--------------+--------------+-----------+------------+
| ID|               Name|Age_Cleaned|Gender|Blood Type|Medical_Condition|Date of Admission|            Hospital|Insurance Provider|BillingAmount_Cleaned|Admission Type|Discharge Date| Medication|Test Results|
+---+-------------------+-----------+------+----------+-----------------+-----------------+--------------------+------------------+---------------------+--------------+--------------+-----------+------------+
|  1|Jacqueline Marshall|         27|  Male|        A+|     Hypertension|       2024-02-22|        Myers-Monroe|  UnitedHealthcare|    39615.60065570468|        Urgent|    2024-02-24| Penicillin|Inconclusive|
|  2|      Robert Sexton|         54|  Male|        A-|           Cancer|       2020-11-19|    Sherman-Martinez|        Blue Cross|    3771.237165573879|        Urg

In [0]:
from pyspark.sql.functions import col
output_df_final = output_df_final.withColumn("BillingAmount_Cleaned", col("BillingAmount_Cleaned").cast("integer"))

In [0]:
from pyspark.sql.functions import col, to_date

output_df_final = output_df_final.withColumn(
    "Date of Admission", 
    to_date(col("Date of Admission"), "yyyy-MM-dd")
)

output_df_final = output_df_final.withColumn(
    "Discharge Date", 
    to_date(col("Discharge Date"), "yyyy-MM-dd")
)

In [0]:
output_df_final.printSchema()

root
 |-- ID: long (nullable = false)
 |-- Name: string (nullable = true)
 |-- Age_Cleaned: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Blood Type: string (nullable = true)
 |-- Medical_Condition: string (nullable = true)
 |-- Date of Admission: date (nullable = true)
 |-- Hospital: string (nullable = true)
 |-- Insurance Provider: string (nullable = true)
 |-- BillingAmount_Cleaned: integer (nullable = true)
 |-- Admission Type: string (nullable = true)
 |-- Discharge Date: date (nullable = true)
 |-- Medication: string (nullable = true)
 |-- Test Results: string (nullable = true)



In [0]:
output_df_final1=output_df_final

In [0]:
display(output_df_final1.schema)

StructType([StructField('ID', LongType(), False), StructField('Name', StringType(), True), StructField('Age_Cleaned', StringType(), True), StructField('Gender', StringType(), True), StructField('Blood Type', StringType(), True), StructField('Medical_Condition', StringType(), True), StructField('Date of Admission', DateType(), True), StructField('Hospital', StringType(), True), StructField('Insurance Provider', StringType(), True), StructField('BillingAmount_Cleaned', IntegerType(), True), StructField('Admission Type', StringType(), True), StructField('Discharge Date', DateType(), True), StructField('Medication', StringType(), True), StructField('Test Results', StringType(), True)])

In [0]:
target_container1 = "processed"

output_df_final.write.mode('overwrite').partitionBy('Medical_Condition').parquet(f"abfss://{target_container1}@projectstorageaccount11.dfs.core.windows.net/processed_healthdata")


In [0]:
target_container2 = "backup"
output_df_final1.write.mode('overwrite').parquet(f"abfss://{target_container1}@projectstorageaccount11.dfs.core.windows.net/processed_healthdata")